In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [4]:
### a new web scraping technique

from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/fr/job/R-30982?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

Postuler pour Lead Product Line Manager, Expression

Rechercher des postesPasser la navigationRechercher des postesNIKE, INC. JOBSEmplois temporairesJoin The Talent CommunityNike au quotidienPrésentationAvantages sociauxMarquesPrésentationJordanConverseÉquipesPrésentationSoutien administratifInnovation de pointeAir Manufacturing et InnovationAéronautiqueCommunicationsService clientConceptionNumériqueInstallationsFinance et comptabilitéAffaires gouvernementales et publiquesRessources humainesDonnées et analyseService juridiqueFabrication et ingénierieMarketingMerchandisingPlanificationConfidentialitéAchatsCréation, développement et gestion de produitsService de gestion des magasinsMagasinsVentesImpact social et communautaireMarketing sportsPlanification stratégiqueChaîne d'approvisionnement, distribution et logistiqueDurabilitéTechnologieSitesPrésentationNike WHQSiège de Nike à New YorkEHQ : Hilversum, Pays-BasELC: Laakdal, BelgiumGreater China HQDiversité, égalité et inclusivitéPrésent

### Prompt Templates

Prompt templates help to translate user input and parameters into instructions for a language model. This can be used to guide a model's response, helping it understand the context and generate relevant and coherent language-based output.

Prompt Templates take as input a dictionary, where each key represents a variable in the prompt template to fill in.

Prompt Templates output a PromptValue. This PromptValue can be passed to an LLM or a ChatModel

In [5]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
#type(res.content)

print(res.content)

[
  {
    "role": "Lead Product Line Manager, Expression",
    "experience": "5 years of product experience",
    "skills": [
      "Strong business acuity & strategic decision making",
      "Phenomenal product eye",
      "Strong style sensibility",
      "Trend foresight",
      "Ability to tell compelling stories",
      "Strategic thinking, product line management and franchise management experience",
      "Effective verbal and written communication skills",
      "Exceptional attention to details and deadlines",
      "Strong organizational & influencing skills"
    ],
    "description": "We are hiring for multiple Lead Product Line Manager positions! We seek individuals who will pioneer new spaces for the Brand and who wants to win and values being part of amazing teams."
  },
  {
    "role": "Lead, Womens Lifestyle FW Product Manager Running Expression",
    "experience": "5 years of product experience",
    "skills": [
      "Strong business acuity & strategic decision making

In [12]:
resultat = res.content
type(resultat)

str

In [13]:
### lets transform the str into json

from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(resultat)
json_res

[{'role': 'Lead Product Line Manager, Expression',
  'experience': '5 years of product experience',
  'skills': ['Strong business acuity & strategic decision making',
   'Phenomenal product eye',
   'Strong style sensibility',
   'Trend foresight',
   'Ability to tell compelling stories',
   'Strategic thinking, product line management and franchise management experience',
   'Effective verbal and written communication skills',
   'Exceptional attention to details and deadlines',
   'Strong organizational & influencing skills'],
  'description': 'We are hiring for multiple Lead Product Line Manager positions! We seek individuals who will pioneer new spaces for the Brand and who wants to win and values being part of amazing teams.'},
 {'role': 'Lead, Womens Lifestyle FW Product Manager Running Expression',
  'experience': '5 years of product experience',
  'skills': ['Strong business acuity & strategic decision making',
   'Phenomenal product eye',
   'Strong style sensibility',
   'Tre

In [26]:
len(json_res)

7

In [15]:
wanted_job = json_res[0]

wanted_job

{'role': 'Lead Product Line Manager, Expression',
 'experience': '5 years of product experience',
 'skills': ['Strong business acuity & strategic decision making',
  'Phenomenal product eye',
  'Strong style sensibility',
  'Trend foresight',
  'Ability to tell compelling stories',
  'Strategic thinking, product line management and franchise management experience',
  'Effective verbal and written communication skills',
  'Exceptional attention to details and deadlines',
  'Strong organizational & influencing skills'],
 'description': 'We are hiring for multiple Lead Product Line Manager positions! We seek individuals who will pioneer new spaces for the Brand and who wants to win and values being part of amazing teams.'}

In [16]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df.head(3)

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio


In [17]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore') # with PersistantClient we create a vectordb in our directory
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [21]:
links = collection.query(query_texts=["experience in python"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}]]

In [22]:
links1 = collection.query(query_texts=wanted_job['skills'], n_results=2).get('metadatas', [])
links1

[[{'links': 'https://example.com/angular-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'}],
 [{'links': 'https://example.com/android-tv-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/android-tv-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/vue-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ios-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/typescript-frontend-portfolio'},
  {'links': 'https://example.com/ma

In [23]:
wanted_job['skills']

['Strong business acuity & strategic decision making',
 'Phenomenal product eye',
 'Strong style sensibility',
 'Trend foresight',
 'Ability to tell compelling stories',
 'Strategic thinking, product line management and franchise management experience',
 'Effective verbal and written communication skills',
 'Exceptional attention to details and deadlines',
 'Strong organizational & influencing skills']

In [25]:
prompt_email = PromptTemplate.from_template(
    
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(wanted_job), "link_list": links})
print(res.content)

Subject: Expert Product Line Management Solutions for Your Business

Dear Hiring Manager,

I came across the job description for the Lead Product Line Manager, Expression position at your esteemed organization, and I was impressed by the role's requirements and responsibilities. As a Business Development Executive at AtliQ, I believe our company can provide the necessary expertise to support your product line management needs.

At AtliQ, we have a proven track record of empowering enterprises with tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of experts has extensive experience in developing and implementing AI-powered tools that can help streamline product line management, trend forecasting, and strategic decision-making.

Our capabilities align with the skills required for the Lead Product Line Manager position, including:

* Strong business acuity and strategic decision-making
* Phenomenal product eye an